<a href="https://colab.research.google.com/github/KingAbdulRehman/LangChain/blob/main/LangChain_ToolCalling/LangChain_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Packages

In [ ]:
!pip install langchain-google-genai

In [105]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata
import requests
from langchain_core.tools import tool
from langchain import hub
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.agents.schema import AgentScratchPadChatPromptTemplate
from rich.console import Console
from rich.markdown import Markdown

# Setup API Keys

In [95]:
if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

if "GOOGLE_CUSTOM_SEARCH_API" not in os.environ:
  os.environ["GOOGLE_CUSTOM_SEARCH_API"] = userdata.get("GOOGLE_CUSTOM_SEARCH_API")

# Setup LLM

In [96]:
llm = ChatGoogleGenerativeAI(
  model="gemini-2.0-flash-exp",
  max_tokens=1000,
  max_retries=3,
  top_k=2,
  temperature=0.7
)

In [97]:
# test llm
print(llm.invoke("Hello").content)

Hi there! How can I help you today?


# Create Search Tool

In [98]:
googleDefaultApiUrl = "https://www.googleapis.com/customsearch/v1?"
searchEngineName = "cx=02feed022528149df"
searchApiKey = os.environ["GOOGLE_CUSTOM_SEARCH_API"]

@tool
def searchEngineRequest(searchText: str):
  """Online search the given text"""
  requestUrl = f"{googleDefaultApiUrl}key={searchApiKey}&{searchEngineName}&q={searchText}"
  res = requests.get(requestUrl)

  return res.json()

In [99]:
# test tool
searchEngineRequest("hello world python")

{'kind': 'customsearch#search',
 'url': {'type': 'application/json',
  'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'},
 'queries': {'request': [{'title': 'Google Custom Search - hello world python',
    'totalResults': '48500000',
    'searchTerms': 'hello world python',
    'count': 10,
    'startIndex': 1,
    'inputEncoding': 'utf8',
    'outputEncoding': 'utf8',
    's

# Create Tool Calling Agent

In [100]:
tools = [searchEngineRequest]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(
  tools=tools,
  llm=llm,
  prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [107]:
ai_search_res = agent_executor.invoke({"input": "Did you know about LA fire incident? please also let me know how many times ago it's starts"})



> Entering new AgentExecutor chain...

Invoking: `searchEngineRequest` with `{'searchText': 'LA fire incident start time'}`


{'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - LA fire incident start time', 'totalResults': '506000000', 'searchTerms':

In [108]:
console = Console()
console.print(Markdown(ai_search_res["output"]))

Yes, I am aware of the recent fire incidents in Los Angeles. There are a few fires that have occurred, but the most
recent one seems to be the Palisades Fire which started on January 7, 2025. There's also the Eaton Fire which also 
started on January 7, 2025. The Palisades Fire has had multiple updates, with the most recent update on January 10,
2025, at 1:57 AM.